In [0]:
dbutils.widgets.text("index", "0")
dbutils.widgets.text("partitions", "1")

In [0]:
!pip install johnnydep
!pip install tqdm
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached johnnydep-1.20.3-py2.py3-none-any.whl (20 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached wimpy-0.6-py2.py3-none-any.whl (3.3 kB)
  Using cached structlog-23.2.0-py3-none-any.whl (62 kB)
  Using cached oyaml-1.0-py2.py3-none-any.whl (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.2 MB/s eta 0:00:00
  Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 22.0
    Not uninstalling packaging at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-47c0a5a7-6acd-4740-a2c6-e13245d5e2d0
    Can't uninstall 'packaging'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the f

In [0]:
import pandas as pd
pypi_df = pd.read_csv("/dbfs/hackathon/data/software_mentions/updated/linked/normalized/pypi_df.csv")
pypi_by_download = pd.read_csv("bq-results-20231026-124638-1698324433695.csv")



In [0]:
join_tables = pypi_df.set_index('mapped_to').join(pypi_by_download.set_index('project_name'))
sorted_by_downloads = join_tables.sort_values("num_downloads", ascending=False)
list_of_projects = list(sorted_by_downloads.index)

In [0]:
# Remove already processed
import os
import json

def load_pypi_map(name):
    if os.path.exists(name):
        with open(name, "r") as f:
            return json.load(f)
    return {}

def load_errors(name):
    if os.path.exists(name):
        with open(name, "r") as f:
            return set(json.load(f))
    return set()

pypi_map = load_pypi_map("pypi_importmap.json")
errors = load_errors("pypi_importmap_errors.json")


list_of_projects = [x for x in list_of_projects if x not in pypi_map if x not in errors]

In [0]:
partititions = int(dbutils.widgets.get("partitions"))
notebook_index = int(dbutils.widgets.get("index"))

projects_in_partition = [x for i, x in enumerate(list_of_projects) if i % partititions == notebook_index]
len(projects_in_partition)

1973

In [0]:
import tqdm
from johnnydep import JohnnyDist

pypi_map = load_pypi_map(f"pypi_importmap-{notebook_index}-{partititions}.json")
version_map = load_pypi_map(f"pypi_versionmap-{notebook_index}-{partititions}.json")
errors = load_errors(f"pypi_importmap_errors-{notebook_index}-{partititions}.json")

retry_errors = False
save_every = 100

count = 0
for pypi_name in tqdm.tqdm(projects_in_partition):
    if pypi_name in pypi_map:
        continue
    if pypi_name in errors:
        if retry_errors:
            errors.remove(pypi_name)
        else:
            continue
    try:
        dist = JohnnyDist(pypi_name)
        pypi_map[pypi_name] = dist.import_names
        version_map[pypi_name] = dist.version_latest_in_spec
    except:
        errors.add(pypi_name)
    count += 1
    if count % save_every == 0:
        with open(f"pypi_importmap-{notebook_index}-{partititions}.json", "w") as f:
            json.dump(pypi_map, f)
        with open(f"pypi_versionmap-{notebook_index}-{partititions}.json", "w") as f:
            json.dump(version_map, f)
        with open(f"pypi_importmap_errors-{notebook_index}-{partititions}.json", "w") as f:
            json.dump(list(errors), f)
        


In [0]:
print(1)

1


In [0]:
with open(f"pypi_importmap-{notebook_index}-{partititions}.json", "w") as f:
    json.dump(pypi_map, f)
with open(f"pypi_versionmap-{notebook_index}-{partititions}.json", "w") as f:
    json.dump(version_map, f)
with open(f"pypi_importmap_errors-{notebook_index}-{partititions}.json", "w") as f:
    json.dump(list(errors), f)